<a href="https://colab.research.google.com/github/trainocate-japan/extending_genai_with_langchain/blob/main/chapter3/langchain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 0: ハンズオンの準備
---

## 必要なライブラリのインストール

In [ ]:
!pip install -q langchain langchain-openai langchain-community

In [ ]:
# !pip install -q langchain==0.2.14 langchain-openai==0.1.22 langchain-community==0.2.12 langchain-core==0.2.33 langchain-text-splitters==0.2.2

In [ ]:
!pip freeze | grep langchain

## API キーの設定
*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `OPENAI_API_KEY` と入力し、その [**値**] に指定されたキーを入力します。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `LANGCHAIN_API_KEY` と入力し、その [**値**] に LangSmith で作成してコピーしておいた API キーを入力します。
*  設定した 2 つのシークレットの [**ノートブックからのアクセス**] を有効にします。
*  入力が完了したら、下のセルを実行します。

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Section 1: LangChain の基本
---

## LLMs
https://python.langchain.com/v0.2/docs/concepts/#llms  

[langchain_openai.llms.base.OpenAI](https://python.langchain.com/v0.2/api_reference/openai/llms/langchain_openai.llms.base.OpenAI.html)

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

result = llm.invoke("自己紹介してください。")
print(result)



*   LLM モデルは文字列を入力として受け取り、文字列を出力する。
*   単純な一問一答の場合に使用する。
*   この程度であれば、LnagChain を使う意味はあまりない。



## Chat models
https://python.langchain.com/v0.2/docs/concepts/#chat-models

[langchain_openai.chat_models.base.ChatOpenAI](https://python.langchain.com/v0.2/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

In [ ]:
messages = [
    SystemMessage(content="以下の日本語を英語に翻訳してください。"),
    HumanMessage(content="こんにちは!"),
]

response = model.invoke(messages)
response

*   Chat Model は Message のリストを入力として受け取り、Message を出力する。
*   Message は `role` `content` `response_metadata` を属性として持つ。
*   LangChain の `SystemMessage` `HumanMessage` `AIMessage` は、それぞれ Chat Complettion API の `role: "system"` `role: "user"` `role: "assistant"` に対応している。

```
{
  "model": "gpt-4o-mini"
  "messages": [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    {"role": "assistant": "content": "こんにちは、ジョンさん！どのようにお手伝いできますか？"},
    {"role": "user": "content": "私の名前がわかりますか？"}
  ],
  (省略)
}
```



### `response` の内容を表示する

In [ ]:
print(response)

### `content` だけを表示する

In [ ]:
print(response.content)

### 会話の履歴を入力に含める

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = model.invoke(messages)
print(result.content)



*   `AIMessage` を入力に含めることができる。
*   AI の回答も含めた過去の会話履歴を入力に含めることによって、会話を継続することができる (後述)。



OpenAI 以外にも、様々な Chat model を利用することができる。  
https://python.langchain.com/v0.2/docs/integrations/chat/

## Prompt templates
プロンプトをテンプレート化しておき、ユーザーの入力で補完してプロンプトを作成する際に使用する。  
https://python.langchain.com/v0.2/docs/concepts/#prompt-templates


### PromptTemplate
[langchain_core.prompts.prompt.PromptTemplate](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """以下の料理のレシピを考えてください。

料理名: {dish}"""

prompt_template = PromptTemplate(
   input_variables=["dish"],
   template=template,
)

prompt = prompt_template.format(dish="カレー")
print(prompt)

`from_template` メソッドを使って以下のように書くこともできる。

In [ ]:
prompt_template = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")

prompt = prompt_template.format(dish="カレー")
print(prompt)

In [ ]:
result = llm.invoke(prompt)
print(result)

*   PromptTemplate はプロンプトを文字列で生成するため、LLM に入力していることに注意。
*   Chat Model に直接入力することはできない。



### ChatPromptTemplate
*  PromptTemplate を Chat model の形式に対応させたもの。  
*  SystemMessage、HumanMessage、AIMessage をそれぞれテンプレート化して、ChatPromptTemplate というクラスでまとめて扱うことができる。

In [ ]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："),
    HumanMessagePromptTemplate.from_template("{question}")
])

messages = chat_prompt.format_prompt(question="簡単な Hello World のコードを作成してください。").to_messages()

print(messages)

以下のように書くこともできる。

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

messages = chat_prompt.format_prompt(question="簡単な Hello World のコードを作成してください。").to_messages()

print(messages)

ChatPromptTemplate によって Message のリストが出力されるので、それを Chat Medel の入力として使用することができる。



In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
result = model.invoke(messages)
print(result.content)

HumanMessage だけにまとめてしままう場合は、`from_template` メソッドを使用することもできる。  
この場合は、コンテキストも HumanMessage のなかに含まれる。

In [ ]:
chat_prompt = ChatPromptTemplate.from_template("""あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：

{question}""")

messages = chat_prompt.format_prompt(question="簡単な Hello World のコードを作成してください。").to_messages()
print(messages)

In [ ]:
result = model.invoke(messages)
print(result.content)

## Output parsers
https://python.langchain.com/v0.2/docs/concepts/#output-parsers

### StrOutputParser
Chat model の出力をテキストに変換する (AIMessage の `content` だけを抽出する) Output parser
[langchain_core.output_parsers.string.StrOutputParser](https://python.langchain.com/v0.2/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
answer = parser.invoke(result)
print(answer)

# Section 2: Chain と LCEL
---

ここまでのコードの書き方では、各ステップの結果物を変数に入れ次のステップに渡す必要があり、コードの量が増えてしまう。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model のインスタンスを作成
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt template のインスタンスを作成
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])
# 完全なプロンプトを作る
messages = chat_prompt.format_prompt(question="簡単な Hello World のコードを作成してください。").to_messages()

# Output parser のインスタンスを作成
parser = StrOutputParser()

# Chat model にプロンプトを入力して結果を得る
result = model.invoke(messages)

# Chat model の出力を Output parser に入力して変換する
answer = parser.invoke(result)
print(answer)

LangChain では、その名が示す通り、Chain という形で複数のステップをつなぎ合わせて実行することができる。
Chain 同士をつなぎ合わせて Chain を作ることもできるため、単にモデルから出力を得て終わりではなく、処理を連鎖的につないで実行することができる。

## Runnables
* LangChain の主要コンポーネントは Runnable プロトコル (インターフェース) を実装しており、いくつかの共通のメソッドで操作できるようになっている。
* Runnable プロトコルを実装しているコンポーネントのオブジェクト自体を Runnable と呼ぶこともある。。



In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])
messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(messages)

In [ ]:
result = model.invoke(messages)
print(result)

In [ ]:
answer = parser.invoke(result)
print(answer)

In [ ]:
chain = chat_prompt | model | parser
answer = chain.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(answer)

## LCEL (LangChain Expression Language) による Chain の記述

*   現在の LangChain では LCEL という記法でコードを記述することが推奨されている。
*   LCEL では Runnable をパイプ (`|`) でつないで Chain を記述することができる。
*   Chain もまた Runnable であり、`invoke` メソッドで実行することができる。



先ほどのコードは LCEL を使って以下のよう書くことができる。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model のインスタンスを作成
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt template のインスタンスを作成
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser のインスタンスを作成
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行する
answer = chain.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(answer)

## Streaming による出力
*   モデルが長い文章を回答する場合、モデルが最後まで回答し終えてから出力すると、ユーザーしばらく待たされることになりユーザビリティが下がる。
*   streaming で出力することにより、モデルからの回答を順次出力していくことが可能になる。
*   Runnables は `stream` や `astream` メソッドをサポートしており、これを使用するとでモデルの回答を streaming で出力することができる。

https://python.langchain.com/v0.2/docs/tutorials/chatbot/#streaming


### streamning しない場合


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# プロンプトのテンプレートを作る
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは AI や機械学習の高度なスキルをもつプロフェッショナルのデータサイエンティストです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行する
answer = chain.invoke({"question": "AI と機械学習、ディープラーニングの関係性をわかりやすく1000文字程度で説明してください。"})
print(answer)

### streaming で出力する場合
[How to stream runnables](https://python.langchain.com/v0.2/docs/how_to/streaming/)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# プロンプトのテンプレートを作る
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは AI や機械学習の高度なスキルをもつプロフェッショナルのデータサイエンティストです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行して streaming で出力する
for chunk in chain.stream({"question": "AI と機械学習、ディープラーニングの関係性をわかりやすく1000文字程度で説明してください。"}):
  print(chunk, end='', flush=True)


## LCEL を理解するための How-to ガイド
https://python.langchain.com/v0.2/docs/how_to/#langchain-expression-language-lcel

# Secton 3: Chat history
---
https://python.langchain.com/v0.2/docs/concepts/#chat-history



*   ここまではモデルとの一問一答での対話だったが、チャットでは会話の継続性が必要になる。
*   ただ、生成 AI モデルの API はステートレスであり、モデル側に会話の履歴を記憶させることはできない。
*   しかし、Chat history を使用して会話の履歴をプロンプトに追加していくことで、会話に継続性をもたせることができるようになる。



### 会話履歴がない場合

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chat template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

以下のセルを実行して、以前の入力や回答に言及する継続的な会話をしてみましょう。

In [ ]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = chain.invoke({"input": q})
    print(f'\nAnswer: {answer}')

## LangChain の記憶 (会話履歴) に関する機能

### Chat History - BaseChatMessageHistory を継承したクラスたち
[langchain_community.chat_message_histories](https://python.langchain.com/v0.2/api_reference/community/chat_message_histories.html)  

#### 例：
`ChatMessageHistory`, `SQLChatMessegeHistory`, `RedisChatMessageHistory`, `DynamoDBChatMessageHistory`, etc.  


#### 概要：
*  決まった形式で会話履歴をデータベース (やメモリ) に保存する機能を提供する。
*  裏で会話履歴の Message を保存するリストを持っていて、そのリストに対する Message の追加や読み出しができる。
*  データベースに読み書きする機能がクラスに実装されているため、その部分のコードを書く必要がなく、バックエンドのデータベースが異なっていてもコードとしてはほぼ同じインターフェイスで操作できる。
*  RDB で自前でテーブル設計して会話履歴を保存したい場合は、これらのクラスが実行している処理を自前で実装すればよい。


#### 使い方：
*  `add_user_message`/`add_ai_message` メソッドを使って、自前で履歴を管理する
*  `RunnableWithMessageHistory` と合わせて使用する

---
### Memory - BaseMemory を継承したクラスたち
[langchain.memory](https://python.langchain.com/v0.2/api_reference/langchain/memory.html)

#### 例：
`ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationSummaryMemory`, etc.

#### 概要：
*  「BaseChatMessageHistory を継承したクラスたち」に対して、会話履歴の一部だけを使用したり、履歴を要約してプロンプトに使用する等の機能を提供する。
*  v0.2 のドキュメントには記載がない (v0.1 のドキュメントには [Beta] と記載されている)


## 会話履歴による会話の継続性の基本的な考え方
ユーザーによる最新の入力だけでなく、過去の会話履歴の Messages をプロンプトに入れてモデルに渡す。  
参考：[How to add memory to chatbots](https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chat template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは人間と会話するチャットボットです。",
        ),
        MessagesPlaceholder(variable_name="messages"), # 複数の Message をプロンプトに挿入するためのプレースホルダ
    ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義
chain = chat_prompt | model | parser

# Chain を実行
answer = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="私の名前はジョンといいます。"
            ),
            AIMessage(content="ジョンさんですね。よろしくお願いします。"),
            HumanMessage(content="私の名前がわかりますか。"),
        ],
    }
)
print(answer)

## Chat history を使う
まず、`ChatMessageHistory` がどのように会話履歴を保持するか確認する。

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

# ChatMessageHistory のインスタンスを作成
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(
    "私の名前はジョンといいます。"
)

demo_ephemeral_chat_history.add_ai_message("ジョンさんですね。よろしくお願いします。")

demo_ephemeral_chat_history.messages

*  `ChatMessageHistory` は `add_user_message` や `add_ai_message` というメソッドを備えており、これを使って会話履歴の Message を追加保存していくことができる。
*  追加した Message は、Message のリストとして保持される。

ユーザーの入力と、それに対するモデルからの回答を順次追加していき、それをプロンプトに含めることによって、会話履歴をもった継続性のある会話が可能となる。

In [ ]:
demo_ephemeral_chat_history = ChatMessageHistory()

# 1 番目の入力
input1 = "私の名前はジョンといいます。"
print(f"INPUT1: {input1}")

demo_ephemeral_chat_history.add_user_message(input1)

# 1 番目の入力に対して回答を出力
response = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)
print(f"ANSWER1: {response}")

# 1 番目の回答を Chat history に追加
demo_ephemeral_chat_history.add_ai_message(response)

# 2 番目の入力
input2 = "私の名前がわかりますか。"
print(f"INPUT2: {input2}")

demo_ephemeral_chat_history.add_user_message(input2)

# 2 番目の入力に対して回答を出力
answer = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)
print(f"ANSWER2: {answer}")

## Chat history を RunnableWithMessageHistory と組み合わせて使用する
*  `RunnableWithMessageHistory` は別の Runnable のために Chat history を管理してくれる Runnable 。
*  別の Runnable (≒ Chain) をラップして、その Chat history の読み出しや更新を行う。
*  `add_user_message` や `add_ai_message` を使ってコードを書かずに済むが、抽象化されてしまっているためややわかりにくく、コードの書き方もクラスの制約を受ける。

[langchain_core.runnables.history.RunnableWithMessageHistory](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 特定のセッション ID に対する ChatMessageHistory のインスタンスを作成
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# Prompt template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      MessagesPlaceholder(variable_name="chat_history"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# RunnableWithMessageHistory のインスタンスを作成
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = runnable_with_history.invoke({"input": q}, config={"configurable": {"session_id": "a123"}})
    print(f'\nAnswer: {answer}')

## SQLChatMessageHistory を使用する
[langchain_community.chat_message_histories.sql.SQLChatMessageHistory](https://python.langchain.com/v0.2/api_reference/community/chat_message_histories/langchain_community.chat_message_histories.sql.SQLChatMessageHistory.html)  
参考：[How to add message history](https://python.langchain.com/v0.2/docs/how_to/message_history/)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_message_histories.sql import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 特定のセッション ID に対する SQLChatMessageHistory のインスタンスを作成
def get_session_history_sql(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# Prompt template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      MessagesPlaceholder(variable_name="chat_history"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# RunnableWithMessageHistory のインスタンスを作成
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history_sql,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = runnable_with_history.invoke({"input": q}, config={"configurable": {"session_id": "a456"}})
    print(f'\nAnswer: {answer}')

In [ ]:
! rm memory.db

Chat history の各クラスを利用して、様々なデータベースに会話履歴を保存することができます。
https://python.langchain.com/v0.2/docs/integrations/memory/  
(ドキュメントのタイトルは Memory ですが、記載されている内容は Chat history です)